In [1]:
import sys
sys.path.append("../")

In [2]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.random import set_seed
import tensorflow.keras.backend as K
from tensorflow.keras import activations

from tensorflow.keras.layers import (
    Input,
    Activation,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Conv2DTranspose,
    Dropout,
    ELU,
    LeakyReLU,
    MaxPool2D,
    PReLU
)

from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau
)

WIDTH, HEIGHT = (256, 256)
BATCH_SIZE = 4
DATAGEN_SEED = 24
NUM_CLASSES = 29

set_seed(DATAGEN_SEED)

2024-01-12 22:41:14.268218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_dir, val_dir = "../cityscapes_data_preprocessed/train/", "../cityscapes_data_preprocessed/val/"
training_images_num, val_images_num = len(os.listdir(train_dir + "img/dummy/")), len(os.listdir(val_dir + "img/dummy/"))
training_images_num, val_images_num

(2972, 500)

In [4]:
from tensorflow.data import Dataset

img_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator()

train_image_datagen = img_datagen.flow_from_directory(train_dir + "img/", class_mode=None, batch_size=BATCH_SIZE, seed=DATAGEN_SEED)
train_mask_datagen = mask_datagen.flow_from_directory(train_dir + "mask/", class_mode=None, batch_size=BATCH_SIZE, seed=DATAGEN_SEED, color_mode="grayscale")
train_mask_generator = Dataset.from_generator(lambda: train_mask_datagen,
    output_types=tf.float32,
    output_shapes=(BATCH_SIZE, 256, 256, 1)
).map(lambda x: tf.reshape(tf.one_hot(tf.cast(x, tf.uint8), depth=NUM_CLASSES), (BATCH_SIZE, 256, 256, NUM_CLASSES)))
train_set = zip(train_image_datagen, train_mask_generator)

val_image_datagen = img_datagen.flow_from_directory(val_dir + "img/", class_mode=None, batch_size=BATCH_SIZE, seed=DATAGEN_SEED)
val_mask_datagen = mask_datagen.flow_from_directory(val_dir + "mask/", class_mode=None, batch_size=BATCH_SIZE, seed=DATAGEN_SEED, color_mode="grayscale")
val_mask_generator = Dataset.from_generator(lambda: val_mask_datagen,
    output_types=tf.float32,
    output_shapes=(BATCH_SIZE, 256, 256, 1)
).map(lambda x: tf.reshape(tf.one_hot(tf.cast(x, tf.uint8), depth=NUM_CLASSES), (-1, 256, 256, NUM_CLASSES)))
val_set = zip(val_image_datagen, val_mask_generator)

Found 2972 images belonging to 1 classes.
Found 2972 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.


2024-01-12 22:41:20.418207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 22:41:20.421853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 22:41:20.422042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 22:41:20.422840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 22:41:20.423027: I tensorflow/compile

In [5]:
def dice_coef(y_true, y_pred, num_classes=29):
    smooth = 1e-15
    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=num_classes)[...,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return K.mean((2. * intersect / (denom + smooth)))

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [6]:
kernel_initializer =  'he_uniform'

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same", kernel_initializer=kernel_initializer)(inputs)
    x = BatchNormalization()(x)
    # x = PReLU()(x)
    x = Activation(activations.mish)(x)
    # x = Activation(modulus_activation)(x)

    x = Conv2D(num_filters, 3, padding="same", kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    # x = PReLU()(x)
    x = Activation(activations.mish)(x)
    
    # x = Activation(modulus_activation)(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same", kernel_initializer=kernel_initializer)(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes=29):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    # s4, p4 = encoder_block(p3, 512)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    # d1 = decoder_block(b1, s4, 512)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(NUM_CLASSES, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

In [7]:
model = build_unet((HEIGHT, WIDTH, 3))
model.compile(optimizer=Adam(learning_rate=1e-3), loss="categorical_crossentropy", metrics=['accuracy', Recall(name="recall"), Precision(name="precision")])

In [8]:
callbacks = [
    ModelCheckpoint(filepath="../models/cityscapes/mish_model.hdf5",
                    monitor="val_loss",
                    save_best_only=True),
    EarlyStopping(monitor="val_loss",
                  patience=5,
                  min_delta=0.01,
                  restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss",
                      factor=0.1,
                      min_delta=0.01
                      patience=2,
                      verbose=1,
                      min_lr=1e-6)
]

In [9]:
history = model.fit(train_set,
                    validation_data=val_set,
                    epochs=300,
                    callbacks=callbacks,
                    steps_per_epoch=training_images_num // BATCH_SIZE,
                    validation_steps=val_images_num // BATCH_SIZE)

Epoch 1/300


2024-01-12 22:41:45.640084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-12 22:41:49.997060: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0c7072abf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-12 22:41:49.997094: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-01-12 22:41:50.000430: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-12 22:41:50.084998: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-01-12 22:41:59.617372: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.27GiB with freed_by_count=0. The call

743/743 [==============================] - ETA: 0s - loss: 1.0980 - accuracy: 0.7045 - recall: 0.6075 - precision: 0.8128

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


743/743 [==============================] - 141s 161ms/step - loss: 1.0980 - accuracy: 0.7045 - recall: 0.6075 - precision: 0.8128 - val_loss: 1.2898 - val_accuracy: 0.6632 - val_recall: 0.6262 - val_precision: 0.7190 - lr: 0.0010
Epoch 2/300
743/743 [==============================] - 120s 161ms/step - loss: 0.8567 - accuracy: 0.7664 - recall: 0.7020 - precision: 0.8477 - val_loss: 0.9116 - val_accuracy: 0.7518 - val_recall: 0.6940 - val_precision: 0.8319 - lr: 0.0010
Epoch 3/300
743/743 [==============================] - 120s 161ms/step - loss: 0.7781 - accuracy: 0.7904 - recall: 0.7339 - precision: 0.8663 - val_loss: 0.8495 - val_accuracy: 0.7700 - val_recall: 0.7201 - val_precision: 0.8427 - lr: 0.0010
Epoch 4/300
743/743 [==============================] - 119s 160ms/step - loss: 0.7198 - accuracy: 0.8049 - recall: 0.7526 - precision: 0.8773 - val_loss: 0.7577 - val_accuracy: 0.7941 - val_recall: 0.7424 - val_precision: 0.8715 - lr: 0.0010
Epoch 5/300
743/743 [=======================

KeyboardInterrupt: 